In [177]:
from icecream import ic
import pandas as pd
import numpy as np
import os
import re
from icecream import ic


basepath = os.getcwd()
raw_txt_dir = "data"
full_path = os.path.join(basepath, raw_txt_dir)

def parsefile(file):
    try:
        with open(file, mode="r") as f:
            contents = f.read()
            return contents
    except Exception as e:
        print(e)    
        

def split_stream(stream):
    try:
        split_reviews = stream.split("\n")
        review_tuples = []
        for sent in split_reviews:
            sent = sent.strip()
            if sent=="":
                continue
            split_sent = sent.split("##")
            assert len(split_sent)==2, f"This sentence review does not split into two: {sent}"
            split_sent[0] = split_sent[0].strip()
            if split_sent[0] == "":
                split_sent[0] = "no_sentiment[0]"
            review_tuples.append((split_sent[0], split_sent[1]))
        return review_tuples 
    except AssertionError as e:
        print(e) 
    

def remove_annotation(stream:str) -> str:
    pattern = r"\*(.*)\*"
    match = re.search(pattern, stream, re.DOTALL)
    if match:
        stream = stream[:match.start()] + stream[match.end():]
        return stream
    return stream

def remove_titles(stream:str) -> str:
    pattern = r"(\[t\].*)"
    return recurse_remove(stream=stream, pattern=pattern)

def recurse_remove(stream, pattern):
    match = re.search(pattern, stream)
    if match is None:
        return stream
    else:
        stream = stream[:match.start()] + stream[match.end():]
        return recurse_remove(stream, pattern)

def preprocess_pipeline(filepath:str) -> list[str]:
    try:
        raw_text = parsefile(filepath)
        clean_of_annotations_text = remove_annotation(stream=raw_text)
        clean_of_review_titles_text = remove_titles(stream=clean_of_annotations_text)
        tuples = split_stream(clean_of_review_titles_text)
        return tuples
    except AssertionError as e:
        print(e)

def read_files():
    for folder in os.listdir(full_path):
        try:
            for file in os.listdir(os.path.join(full_path, folder)):
                if str(file) != "Readme.txt" and str(file) != ".DS_Store":
                    try:
                        preprocessed_tuples = preprocess_pipeline(os.path.join(full_path, folder, file))
                        return preprocessed_tuples
                    except Exception as e:
                        print(e)
        except NotADirectoryError:
            pass

path = "/Users/jossinger/Dropbox/Studies/Bath_Artificial_Intelligence/Course Material/6_NLP/Programming/Submission/data/Reviews-9-products/norton.txt"
preprocess_pipeline(path)[:10]



[('software[-3]',
  'I have read the installation instructions for both NIS 2004 and NAV 2004 prior to installation, but still ended up with the same result...junk software.'),
 ('no_sentiment[0]',
  'Why is it that I can install any other type of software and it installs and works properly?'),
 ('Norton products[-3]',
  'But if I installed either one of these Norton products, neither works after installation?'),
 ('McAfee Anti-Virus 8[+2]]',
  'It can not be the computer or the owner, since I purchased McAfee Anti-Virus 8 and it installs and works fine with no problems.'),
 ('Norton[-2]',
  'I have used Norton for the past 5 years and for the last 2 years, the software has gotten more and more disgraceful.'),
 ('Norton[-2]', 'I am glad that I do not work for Norton.'),
 ('no_sentiment[0]',
  'I have used Norton products in the past and I am familiar with them.'),
 ('no_sentiment[0]', 'I bought NIS 2004 recently to try it out.'),
 ('Installation[-2]', 'After 4 attempted installs, I hav